In [76]:
# Import libraries
import pandas as pd
import numpy as np
# import boto3
# import awswrangler as wr  
import os
# import itertools
import plotly.express as px
# from tqdm import tqdm
# import seaborn as sns

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import matplotlib.pyplot as plt

import plotly.io as pio
pio.renderers.default = 'notebook'

import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
# import sys
# sys.path.append("/home/sesamm/miniconda3/envs/alerting/lib/python3.12/site-packages")

# import datasets

# ML energy

 - Dataset size = 260 rows.
 - 3 types of hardware : A40, A100, V100
 - 4 task variations : chat, chat-concise, instruct, instruct-concise
 - 23 LLM models : 3, 7, 12 and 13B size models. Half are 7b and only one 3b (fastchat)
 -  energy_per_token : wh/token

 => 12 tests / model, basides Llama tested only on A40.
 
-  A100 > V100 >> A40, but not for 3B models

- Whatever the GPU, consumption is a growing function of the models size.

- Without surprise concise models generate less energy due to shorter responses, instruct-concise the lowest


In [3]:
os.listdir('./data/ml_energy/')

['score.csv',
 'V100_instruct_benchmark.csv',
 'models.json',
 'A40_chat_benchmark.csv',
 'A40_chat-concise_benchmark.csv',
 'A100_instruct-concise_benchmark.csv',
 'A100_instruct_benchmark.csv',
 'A40_instruct-concise_benchmark.csv',
 'A40_instruct_benchmark.csv',
 'V100_instruct-concise_benchmark.csv',
 'A100_chat_benchmark.csv',
 'V100_chat-concise_benchmark.csv',
 'A100_chat-concise_benchmark.csv',
 'V100_chat_benchmark.csv']

In [4]:
with open('./data/ml_energy/models.json', 'r') as fd:
    model_params = json.load(fd)

In [5]:
# model_params

In [6]:
len(model_params.keys())

23

In [7]:
score_df = pd.read_csv('./data/ml_energy/score.csv') 
# score_df

In [8]:
ml_energy_df = pd.read_csv('./data/aggregated_ml_energy.csv', keep_default_na=False)

In [9]:
ml_energy_df

,model,throughput,response_length,latency,energy,gpu,task,parameters_count,energy_per_token,dtype,optimization,quantization,cuda-fp16
0,BAIR/koala-13b,25.501170,258.854265,10.063551,1946.198744,A100,chat-concise,13,0.002088,float16,None,None,False
1,BAIR/koala-7b,32.927217,247.045668,7.462044,1195.704790,A100,chat-concise,7,0.001344,float16,None,None,False
2,RWKV/rwkv-raven-7b,64.478527,209.478845,2.921289,736.655330,A100,chat-concise,7,0.000977,float16,None,None,False
3,FreedomIntelligence/phoenix-inst-chat-7b,54.276457,233.511081,4.103907,1002.328573,A100,chat-concise,7,0.001192,float16,None,None,False
4,camel-ai/CAMEL-13B-Combined-Data,26.300221,282.480188,10.727640,2349.183088,A100,chat-concise,13,0.002310,float16,None,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,metaai/llama-7B,23.642990,110.302552,3.730666,636.401785,V100,instruct,7,0.001603,float16,None,None,False
256,tatsu-lab/alpaca-7B,31.257538,127.870047,3.985343,671.828829,V100,instruct,7,0.001459,float16,None,None,False
257,BAIR/koala-7b,31.414144,253.603761,7.832867,1181.363735,V100,instruct,7,0.001294,float16,None,None,False
258,RWKV/rwkv-raven-7b,39.939984,246.796508,5.837914,1302.030667,V100,instruct,7,0.001465,float16,None,None,False


In [10]:
# 23 LLMs, most of which are 7B parameters. One 3B model parameter = lmsys/fastchat-t5-3b-v1.0
pd.DataFrame(ml_energy_df.groupby("parameters_count").model.nunique())

,model
parameters_count,
3,1
7,13
12,2
13,7


In [11]:
# Tests only on A40 for LLaMa
pd.DataFrame(ml_energy_df.model.value_counts()).merge(pd.DataFrame(ml_energy_df.groupby('model').gpu.unique()), left_index=True, right_index=True, how='outer')

,count,gpu
model,,
BAIR/koala-13b,12,"[A100, A40, V100]"
BAIR/koala-7b,12,"[A100, A40, V100]"
FreedomIntelligence/phoenix-inst-chat-7b,12,"[A100, A40, V100]"
Neutralzz/BiLLa-7B-SFT,12,"[A100, A40, V100]"
OpenAssistant/oasst-sft-1-pythia-12b,12,"[A100, A40, V100]"
RWKV/rwkv-raven-7b,12,"[A100, A40, V100]"
Salesforce/xgen-7b-8k-inst,12,"[A100, A40, V100]"
StabilityAI/stablelm-tuned-alpha-7b,12,"[A100, A40, V100]"
camel-ai/CAMEL-13B-Combined-Data,12,"[A100, A40, V100]"


In [12]:
# ml_energy_df.dtype.unique()

In [13]:
ml_energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260 entries, 0 to 259
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   model             260 non-null    object 
 1   throughput        260 non-null    float64
 2   response_length   260 non-null    float64
 3   latency           260 non-null    float64
 4   energy            260 non-null    float64
 5   gpu               260 non-null    object 
 6   task              260 non-null    object 
 7   parameters_count  260 non-null    int64  
 8   energy_per_token  260 non-null    float64
 9   dtype             260 non-null    object 
 10  optimization      260 non-null    object 
 11  quantization      260 non-null    object 
 12  cuda-fp16         260 non-null    bool   
dtypes: bool(1), float64(5), int64(1), object(6)
memory usage: 24.8+ KB


In [14]:
ml_energy_df.describe()

,throughput,response_length,latency,energy,parameters_count,energy_per_token
count,260.000000,260.000000,260.000000,260.000000,260.000000,260.000000
mean,27.608889,224.494313,8.987958,1759.911630,9.030769,0.002202
std,9.312289,71.271262,4.427355,925.369666,3.079233,0.000936
min,14.438333,60.651444,1.986240,335.256588,3.000000,0.000675
25%,21.429872,181.464322,5.534973,1103.811052,7.000000,0.001444
50%,26.743612,242.940060,8.510843,1571.847936,7.000000,0.002130
75%,31.829087,273.593267,11.585607,2217.752910,13.000000,0.002577
max,66.070161,428.193418,24.003582,4466.796723,13.000000,0.004626


In [ ]:
## Dataset used as LLM input for experimentation 

In [192]:
ml_energy_df.groupby(['parameters_count', 'gpu']).energy_per_token.mean().unstack()

gpu,A100,A40,V100
parameters_count,,,
3,0.001303,0.001413,0.000831
7,0.001340,0.002236,0.001484
12,0.002176,0.004248,0.002724
13,0.002268,0.004075,0.002702


In [16]:
# ml_energy_df.groupby(['parameters_count', 'gpu']).size().unstack()
ml_energy_df.groupby(['parameters_count', 'gpu']).model.nunique().unstack()

gpu,A100,A40,V100
parameters_count,,,
3,1,1,1
7,12,13,12
12,2,2,2
13,6,7,6


A100 > V100 > A40

Whatever the GPU, consumption is a growing function of the models size.

Without surprise concise models generate less energy due to shorter responses, instruct-concise the lowest

In [194]:
ml_energy_df.groupby(['gpu', 'task']).energy.mean().unstack()

task,chat,chat-concise,instruct,instruct-concise
gpu,,,,
A100,1434.099724,1344.188430,1384.503047,1118.089412
A40,2580.923048,2481.574307,2509.738636,2039.381988
V100,1591.504919,1552.816582,1541.962225,1295.207577


In [114]:
ml_energy_df.groupby(['gpu', 'task']).energy_per_token.mean().unstack()

task,chat,chat-concise,instruct,instruct-concise
gpu,,,,
A100,0.001718,0.001670,0.001706,0.001637
A40,0.002939,0.002952,0.002940,0.002907
V100,0.001913,0.001955,0.001911,0.001897


In [41]:
ml_energy_df.groupby(['parameters_count', 'task']).response_length.mean().unstack()

task,chat,chat-concise,instruct,instruct-concise
parameters_count,,,,
3,313.373069,312.117529,357.157376,282.170920
7,230.754824,223.669414,228.848139,201.385521
12,203.917842,196.577905,205.564529,196.455395
13,249.665070,236.233767,236.006645,174.232866


In [19]:
# ml_energy_df_100 = ml_energy_df[ml_energy_df.gpu != "A40"]
# ml_energy_df_100[["gpu", "energy","model"]]

In [ ]:
# for model in ml_energy_df.model.unique():
#     print(model)
#     model_df = ml_energy_df[ml_energy_df.model==model]
#     print(model_df.groupby("gpu").energy.mean())
#     print()

# LLM perf

 - Dataset size = 1944 rows.
 - 2 types of hardware : A100-SXM4-80GB	, NVIDIA GeForce RTX 4090. Most tests on A100.
 - 1 task  : chat
 - 181 LLM models : 0.5-70B size models. Mostly 3B, 7B and 13B.
 -  energy_per_token : wh/token

Filters to be on same conditions as ML energy : no quantization, no optimization, float16 only

After filtering : 59 rows.

In [29]:
llm_perf_df = pd.read_csv('./data/aggregated_llm_perf.csv', keep_default_na=False)
llm_perf_df.shape

(1944, 13)

In [136]:
llm_perf_df.model.nunique()

181

In [43]:
llm_perf_df.head()

,model,throughput,response_length,latency,energy,gpu,task,parameters_count,energy_per_token,dtype,optimization,quantization,cuda-fp16
0,EleutherAI/gpt-neox-20b,27.4,256,9.34,3013.632,NVIDIA A100-SXM4-80GB,chat,20.0,0.00327,float16,None,gptq-4bit+exllama-v1,False
1,EleutherAI/pythia-6.9b-deduped,42.3,256,6.05,1649.664,NVIDIA A100-SXM4-80GB,chat,6.9,0.00179,float16,None,gptq-4bit+exllama-v1,False
2,EleutherAI/pythia-6.7b,39.9,256,6.41,1686.528,NVIDIA A100-SXM4-80GB,chat,6.7,0.00183,float16,None,gptq-4bit+exllama-v1,False
3,EleutherAI/pythia-1.4b-deduped,54.8,256,4.67,1115.136,NVIDIA A100-SXM4-80GB,chat,1.4,0.00121,float16,None,gptq-4bit+exllama-v1,False
4,EleutherAI/gpt-j-6b,30.8,256,8.31,2119.680,NVIDIA A100-SXM4-80GB,chat,6.0,0.00230,float16,None,gptq-4bit+exllama-v1,False


In [35]:
llm_perf_df.dtype.value_counts()

dtype
float16     1545
float32      238
bfloat16     161
Name: count, dtype: int64

In [48]:
llm_perf_df.gpu.value_counts()

gpu
NVIDIA A100-SXM4-80GB      1768
NVIDIA GeForce RTX 4090     176
Name: count, dtype: int64

In [90]:
#
import plotly.express as px

fig = px.histogram(llm_perf_df, x="parameters_count")
fig.update_layout(title = 'Nb of models per parameters_count', template='plotly_white')
fig.show()

In [102]:
llm_perf_df.parameters_count.value_counts()#.head(10)

parameters_count
7.000     347
3.000     122
1.300     101
13.000    100
0.031      96
         ... 
1.610       7
0.430       6
0.169       6
70.000      5
2.780       4
Name: count, Length: 72, dtype: int64

In [49]:
llm_perf_df.quantization.value_counts()

quantization
None                    764
bnb-8bit                262
bnb-4bit                256
awq-4bit+gemm           153
awq-4bit+gemv           148
gptq-4bit+exllama-v1    141
gptq-4bit+exllama-v2    119
gptq-4bit               101
Name: count, dtype: int64

In [32]:
llm_perf_df.optimization.value_counts()

optimization
None                 1536
BetterTransformer     302
FlashAttentionV2      106
Name: count, dtype: int64

In [155]:
# Same setup as ML energy
llm_perf_df_filtered = llm_perf_df[(llm_perf_df.optimization=="None") & (llm_perf_df.quantization=="None") & (llm_perf_df.dtype=="float16") \
                                   & (llm_perf_df.parameters_count.isin([3, 6.9, 6.7, 7, 7.2, 7.24, 12, 13]))
                                  ]
len(llm_perf_df_filtered), len(ml_energy_df)

(59, 260)

In [156]:
llm_perf_df_filtered.head()

,model,throughput,response_length,latency,energy,gpu,task,parameters_count,energy_per_token,dtype,optimization,quantization,cuda-fp16
827,EleutherAI/pythia-6.9b-deduped,40.4,256,6.34,2138.112,NVIDIA A100-SXM4-80GB,chat,6.9,0.00232,float16,None,None,False
828,EleutherAI/pythia-6.7b,40.4,256,6.34,2138.112,NVIDIA A100-SXM4-80GB,chat,6.7,0.00232,float16,None,None,False
844,GeneZC/MiniMA-3B,53.2,256,4.81,1345.536,NVIDIA A100-SXM4-80GB,chat,3.0,0.00146,float16,None,None,False
845,GeneZC/MiniMA-2-3B,54.9,256,4.66,1345.536,NVIDIA A100-SXM4-80GB,chat,3.0,0.00146,float16,None,None,False
848,DevaMalla/llama-base-7b,42.4,256,6.04,2082.816,NVIDIA A100-SXM4-80GB,chat,7.0,0.00226,float16,None,None,False


In [157]:
# No model in common
ml_energy_df.merge(llm_perf_df_filtered, on='model', how='inner')

,model,throughput_x,response_length_x,latency_x,energy_x,gpu_x,task_x,parameters_count_x,energy_per_token_x,dtype_x,...,latency_y,energy_y,gpu_y,task_y,parameters_count_y,energy_per_token_y,dtype_y,optimization_y,quantization_y,cuda-fp16_y


In [164]:
llm_perf_df_filtered.parameters_count = llm_perf_df_filtered.parameters_count.apply(lambda x: round(x))

In [160]:
llm_perf_df_filtered.parameters_count.value_counts()

parameters_count
7     40
3     17
12     2
Name: count, dtype: int64

In [161]:
ml_energy_df.parameters_count.value_counts()

parameters_count
7     148
13     76
12     24
3      12
Name: count, dtype: int64

In [162]:
llm_perf_df_filtered.groupby(['parameters_count', 'gpu']).energy_per_token.mean().unstack()

gpu,NVIDIA A100-SXM4-80GB,NVIDIA GeForce RTX 4090
parameters_count,,
3,0.001637,0.000719
7,0.002234,0.001451
12,NaN,0.002455


In [163]:
ml_energy_df.groupby(['parameters_count', 'gpu']).energy_per_token.mean().unstack()

gpu,A100,A40,V100
parameters_count,,,
3,0.001303,0.001413,0.000831
7,0.001340,0.002236,0.001484
12,0.002176,0.004248,0.002724
13,0.002268,0.004075,0.002702


# Methodology v0

        E(model_size, output_tokens) = A * model_size * output_tokens ; A=1.17e-4 Wh

| GPU   |      A |
|----------|:-------------:|
| V100 |    2.15e-4 |
| RTX 4090 | 2.15e-4 |
| A100 |  2.49e-4 |
| A40 | 3.26e-4 |

In [165]:
a_ml_energy = ml_energy_df["energy_per_token"]/ml_energy_df["parameters_count"]
a_llm_perf_df_filtered = llm_perf_df_filtered["energy_per_token"]/llm_perf_df_filtered["parameters_count"]

In [166]:
a_ml_energy.mean(), a_llm_perf_df_filtered.mean() 

(0.00024904050230992696, 0.00030439911218724567)

In [169]:
final_df = pd.concat([ml_energy_df, llm_perf_df_filtered])
len(final_df)

319

In [180]:
final_df.gpu = final_df.gpu.replace({"NVIDIA A100-SXM4-80GB":"A100"})
final_df.to_csv("ml_energy_llmperf_filtered_2024-02-28.csv")

In [181]:
pd.DataFrame(final_df.gpu.value_counts())

,count
gpu,
A100,113
A40,92
V100,84
NVIDIA GeForce RTX 4090,30


In [182]:
pd.DataFrame(final_df.parameters_count.value_counts())

,count
parameters_count,
7,188
13,76
3,29
12,26


In [197]:
# final_df.groupby(["parameters_count", "gpu"]).size().unstack()

In [205]:
final_df.groupby(["parameters_count", "gpu"]).energy_per_token.mean().unstack()

gpu,A100,A40,NVIDIA GeForce RTX 4090,V100
parameters_count,,,,
3,0.001542,0.001413,0.000719,0.000831
7,0.001593,0.002236,0.001451,0.001484
12,0.002176,0.004248,0.002455,0.002724
13,0.002268,0.004075,NaN,0.002702


- estimate A per GPU

In [202]:
final_df.loc[:, "A"] = final_df["energy_per_token"]/final_df["parameters_count"]
a_v0 = pd.DataFrame(final_df.groupby("gpu").A.mean())
a_v0

,A
gpu,
A100,0.000249
A40,0.000327
NVIDIA GeForce RTX 4090,0.000215
V100,0.000215
